In [1]:
import os
from os import path
import pandas as pd

### Build results dataframe

In [23]:
N = 24
tls = [0.5, 0.6, 0.7]
seeds = [i for i in range(1, 4)]
steps = [(i + 1) * 5000 for i in range(N)]

In [24]:
results_list = []
performance_path_tl = '/misc/vlgscratch4/LecunGroup/nvidia-collab/yairschiff/pytorch-PPUU/models/planning_results'
name_tl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-lambdal=0.2' + \
       '-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=False-learnedcost=False-lambdatl={tl}' + \
       '-seed={seed}-novaluestep{step}.model.log'

for tl in tls:
    for seed in seeds:
        for step in steps:
            file_name = path.join(performance_path_tl, name_tl.format(tl=tl, seed=seed, step=step))
            if os.path.isfile(file_name):
                with open(file_name) as f:
                    lines = f.readlines()
                    for line in lines:
                        splits = line.split("|")
                        ep = int(splits[0].split(":")[-1].split("/")[0].strip())
                        time = int(splits[1].split(":")[1].strip())
                        distance = int(splits[2].split(":")[1].strip())
                        success_or_failure = int(splits[3].split(":")[1].strip())
                        df_line = {
                            'lambda_tl': tl,
                            'seed': seed,
                            'step': step,
                            'ep': ep,
                            'time': time,
                            'distance': distance,
                            'result': success_or_failure
                        }
                        results_list.append(df_line)

In [25]:
results_df = pd.DataFrame(results_list)
cols = ['lambda_tl', 'seed', 'step', 'ep', 'time', 'distance', 'result']
results_df = results_df[cols]

### Filter dataframe

In [29]:
# Set filter variables
tl = 0.6
seed = 2
step = 80000
result = 0  # 1 for successes, 0 for failures

In [30]:
# Perform filter
print(f'List of failed episodes for lambda_tl: {tl}, seed: {seed}, step: {step}:')
print(results_df[(results_df.lambda_tl == tl) &
            (results_df.seed == seed) &
            (results_df.step == step) &
            (results_df.result == result)]['ep'].values)

List of failed episodes for lambda_tl: 0.6, seed: 2, step: 80000:
[  2  14  15  25  29  31  33  39  41  44  46  50  60  73  77  84  93  95
  99 116 118 119 122 125 131 132 135 136 137 143 151 152 164 167 170 171
 172 179 181 192 203 208 214 217 225 226 227 229 233 234 239 255 262 272
 276 278 280 281 289 296 298 307 308 313 314 315 320 323 324 332 333 336
 342 345 353 354 356 366 369 373 386 389 390 400 404 409 416 417 423 424
 433 437 438 439 456 459 461 462 464 465 471 472 473 476 482 488 501 509
 513 517 519 521 524 525 528 532 547 550 561]
